In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline 
from sklearn import linear_model, preprocessing
import os
import random

In [2]:
import keras

Using TensorFlow backend.


In [3]:
import librosa
import librosa.display
import librosa.feature

In [4]:
directory_in_str = ("C:\\Users\\Trisha Kadle\\Documents\\Machine_Learning\\New folder\\yes")
fn = []
directory_in_str1 = ("C:\\Users\\Trisha Kadle\\Documents\\Machine_Learning\\New folder\\no")
fn1 = []

fn2 = []

directory = os.fsencode(directory_in_str) 
for file in os.listdir(directory): 
    filename = os.fsdecode(file)
    if filename.endswith(".wav"): 
        fn.append(os.path.join(directory_in_str, filename))
        
directory1 = os.fsencode(directory_in_str1) 
for file in os.listdir(directory1): 
    filename1 = os.fsdecode(file)
    if filename1.endswith(".wav"): 
        fn1.append(os.path.join(directory_in_str1, filename1))

fn2 = np.concatenate((fn, fn1),axis = 0)

In [5]:
mfcc_vec = np.empty((len(fn2),20,11))

for file in range(len(fn2)):
    y, sr = librosa.load(fn2[file], mono=True, sr=None)
    y = y[::3]
    S = librosa.feature.mfcc(y, sr=16000)
    pad = 11-S.shape[1]
    S = np.pad(S,pad_width=((0,0),(0,pad)), mode='constant')
    mfcc_vec[file] = np.r_[S]
                  
np.save('yes_no.npy', mfcc_vec)


In [6]:
yes_and_no_labels = []
yes_labels = np.full(len(fn), 1)
no_labels = np.full(len(fn1), 0)
yes_and_no_labels = np.concatenate((yes_labels,no_labels),axis = 0)

In [7]:
import sklearn.model_selection
from sklearn.model_selection import train_test_split
X_tr, X_ts, y_tr, y_ts = train_test_split(mfcc_vec, yes_and_no_labels, test_size=0.4, random_state=42)


In [8]:
from keras.utils import to_categorical

X_tr = X_tr.reshape(X_tr.shape[0], 20, 11, 1)
X_ts = X_ts.reshape(X_ts.shape[0], 20, 11, 1)


In [9]:
from keras.models import Model, Sequential
from keras.layers import Dense, Activation

In [10]:
import keras.backend as K
from keras.layers import Conv2D, Dropout, Flatten
from keras import optimizers


model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(20, 11, 1)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))

opt = optimizers.Adam(lr=0.001) # beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 19, 10, 32)        160       
_________________________________________________________________
dropout_1 (Dropout)          (None, 19, 10, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6080)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               778368    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 778,657
Trainable params: 778,657
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(X_tr, y_tr, batch_size=100, epochs=50, verbose=1, validation_data=(X_ts, y_ts))


Train on 2851 samples, validate on 1901 samples
Epoch 1/50
2851/2851 [==============================] - 3s 1ms/step - loss: 0.5045 - acc: 0.7569 - val_loss: 0.3381 - val_acc: 0.8669
Epoch 2/50
2851/2851 [==============================] - 2s 843us/step - loss: 0.3267 - acc: 0.8713 - val_loss: 0.2825 - val_acc: 0.8937
Epoch 3/50
2851/2851 [==============================] - 2s 827us/step - loss: 0.2825 - acc: 0.8881 - val_loss: 0.2657 - val_acc: 0.8943
Epoch 4/50
2851/2851 [==============================] - 2s 831us/step - loss: 0.2508 - acc: 0.9056 - val_loss: 0.2405 - val_acc: 0.9011
Epoch 5/50
2851/2851 [==============================] - 2s 819us/step - loss: 0.2242 - acc: 0.9148 - val_loss: 0.2388 - val_acc: 0.9022
Epoch 6/50
2851/2851 [==============================] - 2s 828us/step - loss: 0.2152 - acc: 0.9183 - val_loss: 0.2158 - val_acc: 0.9106
Epoch 7/50
2851/2851 [==============================] - 2s 807us/step - loss: 0.1885 - acc: 0.9288 - val_loss: 0.2131 - val_acc: 0.9137
Ep

In [27]:
from IPython.display import Audio
import pyaudio
import wave

duration = 1  # seconds
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
DURATION = 1
BLOCKSIZE = 1024
WAVE_OUTPUT_FILENAME = "file.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=BLOCKSIZE)

print("Start Recording")

frames = []

for i in range(0, int(RATE / BLOCKSIZE * DURATION)):
    data = stream.read(BLOCKSIZE)
    frames.append(data)

print("Done")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

fn_samp = "C:\\Anaconda3\\Scripts\\file.wav"
Audio(fn_samp, rate=RATE)

Start Recording
Done


In [28]:
fn_samp = "C:\\Anaconda3\\Scripts\\file.wav"
mfcc_vec = np.empty((1,20,11))

y, sr = librosa.load(fn_samp, mono=True, sr=None)
y = y[::3]
S = librosa.feature.mfcc(y, sr=16000)
pad = 11-S.shape[1]
S = np.pad(S,pad_width=((0,0),(0,pad)), mode='constant')
mfcc_samp = S

sample_reshaped = mfcc_samp.reshape(1, 20, 11, 1)

output = np.around(model.predict(sample_reshaped))

if (output == 1):
    print("You said YES")
if (output == 0):
    print("You said NO")

You said YES
